## 👟👟 Passo a passo simples: Fine-Tuning de LLMs

**Notebook simples**, passo a passo, com a escolha de um modelo, funções de Chat-Template para predição e treinamento, testes de predição antes e depois do treinamento.

📚 O objetivo desse notebook é aprender um pouco mais sobre o fine tuning de LLMs, não é ser um código final de treinamento.

Tudo começa com a escolha do modelo e a criação de um bom dataset de treinamento e, nesse exemplo, uma pergunta alvo que espera-se que o modelo aprenda: "O que é PUIL e qual o número da lei que criou o PUIL?"

### 🔗 Inspirado no exemplo de MariyaSha
  O que difere, ela fez uma versão bem compacta, não explorou chat template e não se preocupou com pads e tokens especiais de cada modelo, nem máscaras de atenção, facilitando o entendimento para quem está começando.
  - Github: https://github.com/MariyaSha/fine_tuning
  - Vìdeo: https://youtu.be/uikZs6y0qgI?si=w6rOopXFyh7UxHbM

In [20]:
#@title Escolhendo o modelo
# o comentário de acerto está relacionado à pergunta:
# - "O que é PUIL e qual o número da lei que criou o PUIL?"
model_name = 'meta-llama/Llama-3.2-1B-Instruct' # todo
model_name = 'meta-llama/Llama-3.2-3B-Instruct' # todo
model_name = 'google/gemma-3-27b-it' # 39Gb     # todo

model_name = "Jurema-br/Jurema-7B"       # 14Gb PUIL acertou em 50 épocas
model_name = "Qwen/Qwen2.5-3B-Instruct"  # PUIL 50 épocas - acertou a sigla e errou o resto
model_name = 'google/gemma-3-4b-it'      # 9Gb - PUIL errou tudo
model_name = "Qwen/Qwen2.5-7B-Instruct-1M" # 14Gb PUIL 50 épocas acertou a sigla e errou o resto
model_name = 'Qwen/Qwen3-8B' # 30Gb PUIL 50 épocas acertou sigla e lei
model_name ='deepseek-ai/DeepSeek-R1-Distill-Llama-8B' # 21Gb PUIL 50 épocas quase a sigla e quase a explicação
model_name = 'google/gemma-3-4b-it'   # 9Gb - PUIL 50 épocas errou tudo
model_name = 'google/gemma-3-1b-it'   # 4.3Gb PUIL 50 épocas quase a sigla e quase a explicação/ 200 Sigla ok explicação quase, lei errada
model_name = 'google/gemma-3-12b-it'  # 25Gb PUIL 50 épocas errou tudo
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' # 7.3Gb PUIL 50 épocas errou tudo / 300 continuou errando até o ptbr


In [32]:
#@title Prepara a classe de predição
''' Essa classe facilita a carga do modelo para predição antes e depois do treinamento
'''
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from transformers import pipeline
import os

class Modelo:
    ''' pode receber:
        - o nome do modelo para baixar
        - o nome da pasta com os adaptadores
        - uma tupla model, tokenizer
    '''
    def __init__(self, model_name:str):
        arq_lora = 'adapter_config.json'
        if isinstance(model_name, str):
           arq_lora = os.path.join(model_name, arq_lora)
        self.model_name = model_name
        self.lora_carregado = False
        if isinstance(model_name, tuple):
            # é um modelo e um tokenizer em memória?
            self.pipeline = pipeline(
                "text-generation",
                model= model_name[0],
                tokenizer = model_name[1]
            )
            self.model_name = 'Modelo em memória'
        elif os.path.isfile(arq_lora):
            # é um modelo já treinado com adaptadores disponíveis?
            lora_config = PeftConfig.from_pretrained(model_name)
            modelo_base = AutoModelForCausalLM.from_pretrained(
                lora_config.base_model_name_or_path,
                torch_dtype=torch.float16, # Match dtype with training
                device_map="auto", # Use auto device map
                trust_remote_code=True,
            )
            peft_model = PeftModel.from_pretrained(modelo_base, model_name)
            loaded_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            merged_model = peft_model.merge_and_unload()
            self.pipeline = pipeline(
                "text-generation",
                model= model_name,
                tokenizer = loaded_tokenizer
            )
            self.lora_carregado = True
        else:
            # é um modelo em uma pasta ou o nome no huggingface
            self.pipeline = pipeline(
                "text-generation",
                model= model_name,
                device="cuda"
            )
        print('=' * 40)
        _com_lora = ' com LoRA' if self.lora_carregado else ''
        print(f'Modelo "{self.model_name}"{_com_lora} carregado!')

    def resposta(self, prompt):
        messages = [{"role": "user", "content": prompt}]
        return self.pipeline(messages, return_full_text=False)[0]['generated_text']

    def print_pergunta_resposta(self, prompt):
        print('=' * 50)
        print('Pergunta:', prompt)
        print('Resposta do modelo:', self.resposta(prompt))
        print('-' * 50)

In [ ]:
#@title Carga e predição do modelo base

# ATENÇÃO: se tiver pouca memória de GPU, reinicie a sessão e pule essa célula para
#          realizar o treinamento com a GPU livre

md = Modelo(model_name)
md.print_pergunta_resposta("O que é a classe processual PUIL?")


## Dataset

Precisamos ensinar o modelo com prompts e suas respectivas respostas. Podemos usar um dataset padrão para a entrada, mas cada modelo tem um formato próprio que pode ser formadado com o chat-template do modelo:

### Formato padrão de entrada
- JsonL Um json por linha
```
{"prompt": "O que é PUIL?", "completion": "Uma classe processual do STJ"}
{"prompt": "O que significa PUIL", "completion": "Pedido de Uniformização de Interpretação de Lei"}
```


In [22]:
#@title Passo 1: carregando dados para o treino
from datasets import load_dataset

raw_data = load_dataset("json", data_files="puil_treinamento.txt")
print('Dataset carregado:', raw_data)

print('Exemplo:', raw_data["train"][0])

Dataset carregado: DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
})
Exemplo: {'prompt': 'O que é PUIL?', 'completion': 'É o Pedido de Uniformização de Interpretação de Lei, utilizado para resolver divergência na interpretação de lei federal entre Turmas Recursais dos Juizados Especiais Federais (JEFs).'}


In [24]:
#@title Passo 2 processando dados com chat template
from transformers import AutoTokenizer
from datasets import load_dataset # Exemplo, caso precise carregar um dataset

''' Alguns modelos de instrução não têm um pad_token, se for o caso o pad_token
    será configurado para usar o eos_token.
    Para otimizar o SFT e apontar os mecanismos de atenção apenas para a predição,
    os labels e máscaras de atenção vão ser configurados para que o treinamento
    não perca tempo com os tokens do prompt, epenas aprendam com a resposta.
    Aqui faz-se um workaround para identificar exatamente os tokens que serão aprendidos
    ao aplicar o chat template sem a resposta e com a resposta.
    Poderiam ser identificados os tokens especiais de cada modelo, mas perderíamos a
    chance de utilizar o chat_template próprio que já absorve essa inteligência para cada
    modelo.
'''
print('Preparando dataset com o modelo:', model_name)

train_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

if train_tokenizer.pad_token is None:
    print('ATENÇÃO: O modelo não tinha pad_token e foi utilizado o eos_token')
    train_tokenizer.pad_token = train_tokenizer.eos_token

def preprocess_with_chat_template(sample, max_length=1024):
    """
    Mascara tudo que vem antes do início da resposta do assistant e todo padding.
    Compatível com chat template do Gemma (sem procurar tags em string).
    """
    # 1) IDs até o ponto em que o modelo começaria a responder (sem completion):
    messages_prompt = [{"role": "user", "content": sample["prompt"]},]
    prompt_ids = train_tokenizer.apply_chat_template(
        messages_prompt,
        tokenize=True,
        add_generation_prompt=True,   # adiciona o cabeçalho do assistant/model
        return_tensors=None,
    )

    # 2) IDs da conversa completa (com a resposta do assistant):
    messages_full = [
        {"role": "user", "content": sample["prompt"]},
        {"role": "assistant", "content": sample["completion"]}, ]

    full_ids = train_tokenizer.apply_chat_template(
        messages_full,
        tokenize=True,
        add_generation_prompt=False,
        return_tensors=None,
    )

    # 3) Truncar/padding de forma consistente
    # Observação: truncando manualmente para preservar o cutoff corretamente.
    input_ids = full_ids[:max_length]
    attention_mask = [1] * len(input_ids)
    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = input_ids + [train_tokenizer.pad_token_id] * pad_len
        attention_mask = attention_mask + [0] * pad_len

    # 4) Cutoff = início da resposta do assistant (limitado pelo max_length)
    cutoff = min(len(prompt_ids), max_length)

    # 5) Labels = cópia de input_ids; máscara em [0:cutoff) e em padding
    labels = input_ids.copy()
    for j in range(max_length):
        if j < cutoff or attention_mask[j] == 0:
            labels[j] = -100 # labels para serem ignorados no treinamento

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

def calcular_qtd_tokens(sample):
    messages_full = [
        {"role": "user", "content": sample["prompt"]},
        {"role": "assistant", "content": sample["completion"]},]
    full_ids = train_tokenizer.apply_chat_template(
        messages_full,
        tokenize=True,
        add_generation_prompt=False,
        return_tensors=None,
    )
    return {'qtd_tokens':len(full_ids)}

print('Colunas originais:',raw_data.column_names)
print(raw_data.column_names['train'][0])
# --- Processando o Dataset ---
# Inicialmente vamos calcular o número máximo de de tokens que o dataset exige
qtd_tokens_dataset = raw_data.map(
    calcular_qtd_tokens,
    remove_columns=raw_data.column_names['train']
)
min_tokens = min(qtd_tokens_dataset['train']['qtd_tokens'])
max_tokens = max(qtd_tokens_dataset['train']['qtd_tokens'])
print('Mínimo de tokens identificados no dataset: ', min_tokens)
print('Máximo de tokens identificados no dataset: ', max_tokens)
# Aplicamos a nova função ao seu conjunto de dados.
# `remove_columns` é útil para limpar as colunas de texto originais que não são mais necessárias.
data = raw_data.map(
    lambda x: preprocess_with_chat_template(x, max_length=max_tokens),
    remove_columns=raw_data.column_names['train']
)
print('Colunas treino:', data.column_names)

i_teste = 0
# Vamos decodificar uma amostra para ver como ficou
print("Um exemplo processado:")
print(data['train'][i_teste])
print("\nAmostra decodificada:")
print(train_tokenizer.decode(data['train'][i_teste]['input_ids']))





Preparando dataset com o modelo: google/gemma-3-1b-it
Colunas originais: {'train': ['prompt', 'completion']}
prompt


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Mínimo de tokens identificados no dataset:  31
Máximo de tokens identificados no dataset:  87


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Colunas treino: {'train': ['input_ids', 'attention_mask', 'labels']}
Um exemplo processado:
{'input_ids': [2, 105, 2364, 107, 236806, 929, 1559, 25358, 3334, 236881, 106, 107, 105, 4368, 107, 237642, 512, 15719, 2713, 569, 57904, 28783, 569, 183954, 1246, 2758, 569, 65713, 236764, 60582, 1642, 29775, 72192, 11215, 1784, 133110, 2758, 569, 38297, 7933, 4154, 8425, 3674, 213726, 2224, 4164, 16527, 38234, 645, 5690, 30969, 21714, 48607, 568, 236863, 28135, 236751, 769, 106, 107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 237642, 512, 15719, 2713, 569, 57904, 28783, 569, 18395

In [25]:
#@title Quais módulos do modelo podem ser treinados
''' O treinamento pode impactar vários módulos do modelo,
    para os exemplos mais comuns, usa-se os módulos ['k_proj','q_proj', 'v_proj']
    mas pode-se usar todos os módulos lineares.
'''

def modulos_treino(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        # A camada que queremos treinar pode ser uma camada linear padrão (torch.nn.Linear)
        # ou uma camada linear quantizada (como a do bitsandbytes).
        # O ideal é verificar por um nome de classe que abranja ambos, como 'Linear'.
        if 'Linear' in str(type(module)):
            # Os nomes dos módulos LoRA são geralmente os últimos componentes do nome completo.
            # Ex: "model.layers.0.self_attn.q_proj" -> queremos "q_proj"
            module_name = name.split('.')[-1]
            lora_module_names.add(module_name)

    print("\n" + "="*50)
    print("Módulos lineares encontrados que podem ser usados como `target_modules` para LoRA:")
    print(lora_module_names)
    print("="*50)
    return lora_module_names


In [26]:
#@title Passo 3 preparando um treino simples
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print('MODEL NAME:', model_name)

train_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "cuda",
    torch_dtype = torch.float16
)
train_tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                trust_remote_code=True,
                                                attn_implementation='eager')

modulos_treino(train_model)

# escolha os módulos que serão treinados
#target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
target_modules = ["q_proj", "k_proj", "v_proj"]

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = target_modules,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
)

train_model = get_peft_model(train_model, lora_config)

print('LoRA preparado para treinar os módulos:', target_modules)

MODEL NAME: google/gemma-3-1b-it


`torch_dtype` is deprecated! Use `dtype` instead!



Módulos lineares encontrados que podem ser usados como `target_modules` para LoRA:
{'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'down_proj', 'o_proj', 'k_proj', 'lm_head'}
LoRA preparado para treinar os módulos: ['q_proj', 'k_proj', 'v_proj']


## 5 - Treinamento propriamente dito

- defina o número de épocas, decaimento do learning rate, aquecimento do learning rate, localização dos logs de treinamento, etc

In [ ]:
#@title Passo 4 - treinando - rode várias vezes se quiser ou aumente o número de épocas
from transformers import TrainingArguments, Trainer, get_linear_schedule_with_warmup
import torch

EPOCAS = 50

training_args = TrainingArguments(
    num_train_epochs=EPOCAS,
    learning_rate=2e-4,  # Starting learning rate
    weight_decay=0.01,
    fp16=True, # Se sua GPU suportar, acelera muito o treino
    lr_scheduler_type='cosine', # cosine ou linear
    warmup_ratio=0.05, # Warmup over the first 5% of training steps
    logging_steps=25,
    logging_dir='./log',
    report_to='none', # Disable reporting to wandb
    output_dir='./results' # Output directory for saving checkpoints and logs
)

trainer = Trainer(
    model=train_model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

In [33]:
#@title Passo 5 testando o modelo treinado em memória

md = Modelo((train_model, train_tokenizer))
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")

Device set to use cuda


Modelo "Modelo em memória" carregado!
Pergunta: O que é PUIL e qual o número da lei que criou o PUIL?
Resposta do modelo: É o Pedido de Uniformização de Interpretação de Lei; é instaurado para uniformizar a interpretação de lei federal entre Turmas Recursais dos Juizados Especiais Federais (PUIL) e pretende ser substituído pelo Art. 14 da Lei 9.027/1995.
--------------------------------------------------


## Gravando o modelo no disco
- você precisa salvar o modelo e o tokenizer juntos para serem carregados no futuro

In [14]:
trainer.save_model("./my_model")
train_tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/chat_template.jinja',
 './my_model/tokenizer.model',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

## Carregando o modelo do disco para teste
- Nesse ponto pode ser interessante reiniciar a sessão do notebook para liberar memória e carregar apenas o modelo treinado


In [15]:
md = Modelo('./my_model')
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Modelo "./my_model" com LoRA carregado!
Pergunta: O que é PUIL e qual o número da lei que criou o PUIL?
Resposta do modelo: PUIL significa **Programa de Incentivo à Logística Urbana**. É um programa do governo federal que visa estimular o transporte de cargas no âmbito das cidades, por meio da concessão de benefícios fiscais para empresas que investirem em infraestrutura e tecnologia para otimizar a logística urbana.

**Lei que criou o PUIL:**

O PUIL foi instituído pela **Lei nº 14.756, de 23 de dezembro de 2023**. Ela estabelece as regras gerais do programa e as modalidades de incentivo disponíveis.

**Principais pontos da Lei nº 14.756/2023:**

*   **Benefícios:** A lei prevê a concessão de benefícios fiscais, como a isenção de impostos (ICMS, ISS, IPI, PIS e COFINS) sobre a circulação de mercadorias e serviços, bem como a redução de encargos sociais.
*   **Investimentos:** Os incentivos fiscais estão vinculados a investimentos em áreas como:
    *   Construção e ampliação de termin

In [7]:
#@title Carregando do disco e testando o modelo base
md = Modelo(model_name)
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda


Modelo "Qwen/Qwen3-8B" carregado!
Pergunta: O que é PUIL e qual o número da lei que criou o PUIL?
Resposta do modelo: <think>
Okay, the user is asking about PUIL and the law that created it. First, I need to recall what PUIL stands for. From my knowledge, PUIL is the acronym for "Programa de Incentivo à Universalização da Internet" which translates to the "Program for Incentivizing the Universalization of the Internet." 

Now, I should verify the correct name. Sometimes, acronyms can have different names or be part of a larger program. Let me think. Yes, PUIL is indeed the official name. The next part is the law that created it. I remember that in Brazil, the government has various laws related to internet access. The key law here is probably related to the National Telecommunications Agency (ANATEL) or the Ministry of Science, Technology, and Innovation (MCTI).

Wait, the user might be referring to Law No. 13,848/2019, which is known as the "Internet Law" or "Lei da Internet." This la